In [1]:
import qiskit
print("qiskit version:", qiskit.__version__)

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.

In [3]:
from qiskit import IBMQ

# Save your IBM Quantum API token
IBMQ.save_account('a80d2cab0d6db0c274e192f348e18e6c40499d0f97008669e43bfb5db60e9084815b9cd4c8f06740e565a04e6e5a0fc09c39566a8fd79abe309f4ad77bedf432')

ImportError: cannot import name 'IBMQ' from 'qiskit' (c:\Users\alfre\OneDrive\Documents\GitHub\DD1366\DA150X\.venv\Lib\site-packages\qiskit\__init__.py)

In [1]:
import qiskit
from qiskit_aer.primitives import SamplerV2
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp, Operator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import EstimatorV2 as Estimator
import numpy as np


# n = Number of qubits, indicies_to_mark = List of indicies
def phase_oracle(n, indicies_to_mark, name='Oracle'):
    #create a circuit with n qubits
    qc = QuantumCircuit(n, name=name)
    # Create a identity matrix
    oracle_matrix = np.identity(2**n)
    # Mark indicies in the identity matrix
    for index_to_mark in indicies_to_mark:
        oracle_matrix[index_to_mark, index_to_mark] = -1
    # Convert identity matrix to a unitary gate and add
    # to the circuit
    qc.unitary(Operator(oracle_matrix), range(n))
    return qc

# Optimization: using a dictionary to save created diffusers
# to avoid having to construct them multiple times
diffuser_dict = {}
# n = number of qubits
def diffuser(n):
    if n not in diffuser_dict:
    # Create a quantum circuit with n qubits
        qc = QuantumCircuit(n, name='Diffuser')
        # Apply hadamar gates to all qubits
        qc.h(range(n))
        # Apply U_f0
        qc.append(phase_oracle(n, [0]), range(n))
        qc.h(range(n))
        diffuser_dict.update({n: qc})
    return diffuser_dict.get(n)


# n = Number of qubits, indicies_to_mark = list of marked items
# r = Number of grover iterations to be performed
def Grover(n, indicies_to_mark, r):
# Create a circuit with n qubits and n classical output bits
    qc = QuantumCircuit(n, n)
    # Apply Hadamar gates to all qubits
    qc.h(range(n))
    # Create oracle and diffuser circuits
    qc_oracle = phase_oracle(n, indicies_to_mark)
    qc_diffuser = diffuser(n)
    # Apply the oracle and diffuser circuits r times
    for _ in range(r):
        qc.append(qc_oracle, range(n))
        qc.append(qc_diffuser, range(n))
    # Measure all qubits and map them onto the clasical bits
    qc.measure(range(n), range(n))
    return qc


def run_grover(n, indicies_to_mark, r, backend, shots):
    # Create the Grover circuit
    grover_circuit = Grover(n, indicies_to_mark, r)
    # Use the Aer qasm simulator
    aer_simulator = qiskit_aer.Aer.get_backend('qasm_simulator')
    # Execute the circuit on the qasm simulator
    job = aer_simulator.run(grover_circuit, shots=shots)
    # Get the result of the simulation
    result = job.result()
    # Get the counts of the result
    counts = result.get_counts()
    return counts


def run_grover_ibm(n, indicies_to_mark, r, backend, shots):
    # Create the Grover circuit
    grover_circuit = Grover(n, indicies_to_mark, r)
    # Use the IBM Quantum Experience
    provider = IBMQ.load_account()
    # Get the least busy backend
    backend = provider.get_backend(backend)
    # Execute the circuit on the least busy backend
    job = backend.run(grover_circuit, shots=shots)
    # Get the result of the simulation
    result = job.result()
    # Get the counts of the result
    counts = result.get_counts()
    return counts

run_grover(3, [0], 1, 'qasm_simulator', 1024)

ImportError: Qiskit is installed in an invalid environment that has both Qiskit >=1.0 and an earlier version. You should create a new virtual environment, and ensure that you do not mix dependencies between Qiskit <1.0 and >=1.0. Any packages that depend on 'qiskit-terra' are not compatible with Qiskit 1.0 and will need to be updated. Qiskit unfortunately cannot enforce this requirement during environment resolution. See https://qisk.it/packaging-1-0 for more detail.